# Imports

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import networkx as nx
from node2vec import Node2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

C:\Users\Ayan\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Ayan\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Ayan\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


# Load data

In [2]:
soc_reddit_body = pd.read_csv('soc-redditHyperlinks-body.tsv', sep='\t')

In [3]:
soc_reddit_body.head()

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08..."
1,theredlion,soccer,1u4qkd,2013-12-31 18:18:37,-1,"101.0,98.0,0.742574257426,0.019801980198,0.049..."
2,inlandempire,bikela,1u4qlzs,2014-01-01 14:54:35,1,"85.0,85.0,0.752941176471,0.0235294117647,0.082..."
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0...."
4,playmygame,gamedev,1u4w5ss,2014-01-01 02:51:13,1,"715.0,622.0,0.777622377622,0.00699300699301,0...."


In [4]:
embed_subreddit = pd.read_csv('web-redditEmbeddings-subreddits.csv')

In [5]:
embed_subreddit.head()

,spiders,0.158972,0.285813,0.226329,-0.183338,-0.163159,-0.18891,0.422705,0.168602,0.178193,...,-0.025041,0.184655,0.111942,-1.125799,0.223129,-0.387323,-0.234882,-0.252297,-0.054315,0.500009
0,askreddit,-0.499114,0.323983,-0.424809,-0.222705,-0.327201,0.412856,1.040088,0.996697,-1.256240,...,0.072151,0.226658,0.535705,0.039861,-0.027087,-0.239952,0.554510,0.676439,-0.328846,0.941547
1,globaloffensivetrade,-0.023145,-1.199374,1.661484,-1.025296,1.424670,-1.179507,-1.035703,0.329301,-0.567272,...,2.998430,-1.235703,-0.326850,4.381238,-1.965378,-2.369674,-0.615389,2.010601,-0.919314,-2.149625
2,fireteams,2.492506,-2.529917,-0.448484,-3.543441,-0.586122,-0.101517,1.837567,-0.215715,-2.460388,...,-1.875214,0.969291,0.497958,2.941132,2.423561,1.315691,-0.959618,0.151845,-0.852492,-0.467526
3,funny,-0.819370,-0.865261,0.301753,0.018787,0.201029,0.070462,0.903895,1.038460,-0.306950,...,1.364194,0.830233,-0.137822,-2.204667,-0.162008,0.390291,-0.556404,0.322684,0.641634,1.320716
4,the_donald,-0.123265,-0.610208,0.361495,-1.171773,0.367407,0.624919,2.420601,-0.447998,1.393862,...,0.981761,1.044754,-2.334628,-0.005856,-1.647531,-0.139574,-0.284726,0.315069,-0.846018,0.800535


In [6]:
nodes = set(soc_reddit_body['SOURCE_SUBREDDIT'].tolist()).union(set(soc_reddit_body['TARGET_SUBREDDIT'].tolist()))

In [7]:
len(nodes)

35776

In [8]:
node_emb_names = []
node_embd_vec = []

for j in tqdm(range(len(embed_subreddit))):
    node_emb_names.append(embed_subreddit.iloc[j].tolist()[0])
    node_embd_vec.append(embed_subreddit.iloc[j].tolist()[1:302])

100%|██████████████████████████████████████████████████████████████████████████| 51277/51277 [00:31<00:00, 1603.46it/s]


In [9]:
node_embd_name_vec = dict(zip(node_emb_names,node_embd_vec))

In [10]:
len(node_emb_names)

51277

In [11]:
len(nodes.intersection(set(node_emb_names)))

21674

In [12]:
node_1_embed = []
node_2_embed = []
year_list = []
found_1 = []
found_2 = []

for index, row in tqdm(soc_reddit_body.iterrows()):
    year_list.append(row['TIMESTAMP'].split()[0].split('-')[0])
    try:
        node_1_embed.append(node_embd_name_vec[row['SOURCE_SUBREDDIT']])
        found_1.append(1)
    except:
        node_1_embed.append([0]*300)
        found_1.append(0)
    try:
        node_2_embed.append(node_embd_name_vec[row['TARGET_SUBREDDIT']])
        found_2.append(1)
    except:
        node_2_embed.append([0]*300)
        found_2.append(0)

print(set(year_list))

286561it [00:40, 7071.62it/s]

{'2014', '2017', '2013', '2015', '2016'}


In [13]:
soc_reddit_body['year'] = year_list
soc_reddit_body['source_embed'] = node_1_embed
soc_reddit_body['target_embed'] = node_2_embed
soc_reddit_body['found_1'] = found_1
soc_reddit_body['found_2'] = found_2

In [14]:
soc_reddit_body

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES,year,source_embed,target_embed,found_1,found_2
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08...",2013,"[-2.785298, -0.166391, 1.592624, -1.269829, 2....","[-1.402933, -1.1156540000000001, 0.921651, -0....",1,1
1,theredlion,soccer,1u4qkd,2013-12-31 18:18:37,-1,"101.0,98.0,0.742574257426,0.019801980198,0.049...",2013,"[-0.239977, -0.24648299999999998, 0.0596789999...","[-2.7988470000000003, -1.243099, 0.513319, 0.1...",1,1
2,inlandempire,bikela,1u4qlzs,2014-01-01 14:54:35,1,"85.0,85.0,0.752941176471,0.0235294117647,0.082...",2014,"[0.070123, -0.001389, 0.143414, -0.46698900000...","[0.15803, 0.035602999999999996, 0.197221, -0.4...",1,1
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0....",2013,"[-0.41374399999999995, 0.13559300000000002, -0...","[-0.5422899999999999, 1.0747719999999998, 0.23...",1,1
4,playmygame,gamedev,1u4w5ss,2014-01-01 02:51:13,1,"715.0,622.0,0.777622377622,0.00699300699301,0....",2014,"[0.006368, -0.135252, 0.45853299999999997, -0....","[-0.033949, 0.15096600000000002, 1.039643, -0....",1,1
...,...,...,...,...,...,...,...,...,...,...,...
286556,negareddit,debatefascism,68im20s,2017-04-30 16:31:26,1,"441.0,405.0,0.775510204082,0.0294784580499,0.0...",2017,"[-0.392242, 0.186111, 0.203193, -0.441168, 0.3...","[-0.202682, 0.126838, -0.077966, -0.2579079999...",1,1
286557,mildlynomil,justnomil,68imlas,2017-04-30 04:19:03,1,"2226.0,1855.0,0.786163522013,0.00224618149146,...",2017,"[-0.008284, -0.075661, 0.082401, -0.049988, 0....","[0.124855, -0.792166, 0.149331, 0.376434999999...",1,1
286558,mmorpg,blackdesertonline,68ip5os,2017-04-30 16:54:08,1,"1100.0,909.0,0.778181818182,0.00181818181818,0...",2017,"[0.008334999999999999, 0.0209, 0.9342010000000...","[-0.272359, -0.833299, 1.339546, -0.268322, 0....",1,1
286559,electricskateboards,askreddit,68ipb2s,2017-04-30 16:41:53,1,"1876.0,1567.0,0.78144989339,0.00692963752665,0...",2017,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.49911400000000006, 0.32398299999999997, -0...",0,1


# Trains on 2017 and test on 2018

In [21]:
train = soc_reddit_body[soc_reddit_body['year'] == '2017']
test = soc_reddit_body[soc_reddit_body['year'] == '2016']
print(len(train),len(test))

36406 98229


In [22]:
train_nodes = set(train['SOURCE_SUBREDDIT'].tolist()).union(set(train['TARGET_SUBREDDIT'].tolist()))
test_nodes = set(test['SOURCE_SUBREDDIT'].tolist()).union(set(test['TARGET_SUBREDDIT'].tolist()))
print('Inductive nodes: ', len(test_nodes.difference(train_nodes)))

Inductive nodes:  12419


In [23]:
inductive_nodes = test_nodes.difference(train_nodes)
inductive_test = test[test['SOURCE_SUBREDDIT'].isin(inductive_nodes) | test['TARGET_SUBREDDIT'].isin(inductive_nodes)]
print('Size of inductive test set: ', len(inductive_test))

Size of inductive test set:  25798


In [24]:
train_valid = train[train['found_1'].isin([1]) & train['found_2'].isin([1])]
print(len(train_valid))
inductive_test_valid = inductive_test[inductive_test['found_1'].isin([1]) & inductive_test['found_2'].isin([1])]
print(len(inductive_test_valid))

31873
17026


In [25]:
train_valid = train_valid[['SOURCE_SUBREDDIT','TARGET_SUBREDDIT', 'year', 'source_embed', 'target_embed']]
inductive_test_valid = inductive_test_valid[['SOURCE_SUBREDDIT','TARGET_SUBREDDIT', 'year', 'source_embed', 'target_embed']]

## Generating negatives

In [26]:
train_valid_node_dict = dict()

for index, row in tqdm(train_valid.iterrows()):
    train_valid_node_dict[row['SOURCE_SUBREDDIT']] = row['source_embed']
    train_valid_node_dict[row['TARGET_SUBREDDIT']] = row['target_embed']
    
inductive_test_valid_node_dict = dict()

for index, row in tqdm(inductive_test_valid.iterrows()):
    inductive_test_valid_node_dict[row['SOURCE_SUBREDDIT']] = row['source_embed']
    inductive_test_valid_node_dict[row['TARGET_SUBREDDIT']] = row['target_embed']

31873it [00:04, 6545.27it/s]
17026it [00:02, 6531.16it/s]


In [27]:
train_neg = pd.DataFrame()

source_list = []
target_list = []
year_list = []
source_embed_list = []
target_embed_list = []

for j in tqdm(range(len(train_valid))):
    
    src = np.random.choice(list(train_valid_node_dict.keys()))
    trgt = np.random.choice(list(train_valid_node_dict.keys()))
    
    source_list.append(src)
    target_list.append(trgt)
    year_list.append(2016)
    source_embed_list.append(train_valid_node_dict[src])
    target_embed_list.append(train_valid_node_dict[trgt])
    
train_neg['SOURCE_SUBREDDIT'] = source_list
train_neg['TARGET_SUBREDDIT'] = target_list
train_neg['year'] = year_list
train_neg['source_embed'] = source_embed_list
train_neg['target_embed'] = target_embed_list

train_all = pd.concat([train_valid,train_neg])
train_all['label'] = [1] * len(train_valid) + [0] * len(train_neg)
train_all

100%|███████████████████████████████████████████████████████████████████████████| 31873/31873 [02:35<00:00, 205.47it/s]


,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,year,source_embed,target_embed,label
232949,islamwatch,islam,2017,"[-0.025353, -0.048428, 0.036367000000000003, -...","[-0.134543, 0.828586, -0.418367, -0.511281, -0...",1
241752,westshire,lordsofminecraft,2017,"[-0.007277, -0.089598, 0.079178, -0.06075, 0.0...","[0.030913, -0.270242, 0.221529, -0.101273, 0.1...",1
243386,timesyncs,writingprompts,2017,"[0.019989, -0.102148, 0.028444999999999998, -0...","[0.764895, -1.11499, -0.031956, -0.834976, 1.0...",1
244010,timesyncs,writingprompts,2017,"[0.019989, -0.102148, 0.028444999999999998, -0...","[0.764895, -1.11499, -0.031956, -0.834976, 1.0...",1
244278,cris9696,androidgaming,2017,"[-0.014596000000000001, -0.09098300000000001, ...","[0.30240300000000003, 0.173463, 0.909001000000...",1
...,...,...,...,...,...,...
31868,portabledabs,shitstatistssay,2016,"[-0.020151, -0.114968, 0.10251500000000001, -0...","[-0.43328999999999995, 0.267779, 0.047897, -0....",0
31869,kenm,zedmains,2016,"[-0.061797000000000005, -0.050186, 0.091866999...","[-0.226698, -0.223068, 0.21559899999999999, -0...",0
31870,random_acts_of_sex,htc,2016,"[-0.041365, -0.341532, 0.152848, -0.064262, 0....","[0.011974, -0.15041600000000002, 0.46375600000...",0
31871,rants,indianauniversity,2016,"[-0.033384, -0.159664, 0.129465, -0.117684, -0...","[-0.036968, -0.175541, 0.147925, -0.18438, 0.0...",0


In [28]:
inductive_test_valid_neg = pd.DataFrame()

source_list = []
target_list = []
year_list = []
source_embed_list = []
target_embed_list = []

for j in tqdm(range(len(inductive_test_valid))):
    
    src = np.random.choice(list(inductive_test_valid_node_dict.keys()))
    trgt = np.random.choice(list(inductive_test_valid_node_dict.keys()))
    
    source_list.append(src)
    target_list.append(trgt)
    year_list.append(2017)
    source_embed_list.append(inductive_test_valid_node_dict[src])
    target_embed_list.append(inductive_test_valid_node_dict[trgt])
    
inductive_test_valid_neg['SOURCE_SUBREDDIT'] = source_list
inductive_test_valid_neg['TARGET_SUBREDDIT'] = target_list
inductive_test_valid_neg['year'] = year_list
inductive_test_valid_neg['source_embed'] = source_embed_list
inductive_test_valid_neg['target_embed'] = target_embed_list

test_all = pd.concat([inductive_test_valid,inductive_test_valid_neg])
test_all['label'] = [1] * len(inductive_test_valid) + [0] * len(inductive_test_valid_neg)
test_all

100%|███████████████████████████████████████████████████████████████████████████| 17026/17026 [01:31<00:00, 186.91it/s]


,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,year,source_embed,target_embed,label
148286,listofsubreddits,changelog,2016,"[0.000697, -0.065453, 0.060517999999999995, -0...","[-0.02054, -0.07195599999999999, 0.074525, -0....",1
148411,faggots,redditrequest,2016,"[0.001968, 0.0008300000000000001, -0.001843, -...","[-0.001361, 0.36938899999999997, 0.024178, -0....",1
149389,tppthegame,twitchplayspokemon,2016,"[-0.095835, -0.069751, 0.116627, -0.004266, 0....","[-1.164833, -0.03354, 0.916997, -0.47464799999...",1
150673,futuristparty,technostism,2016,"[-0.07631399999999999, 0.11399000000000001, 0....","[-0.109224, -0.179844, 0.160706, -0.233548, -0...",1
151190,tallyhall,deepintoyoutube,2016,"[-0.022491, -0.082375, 0.089028, -0.0731, 0.01...","[-0.14813800000000002, -0.21752, 0.468912, -0....",1
...,...,...,...,...,...,...
17021,projectara,glassheads,2017,"[-0.035039999999999995, -0.050692, 0.129063, -...","[0.253877, -0.400094, 0.330165, -0.47161800000...",0
17022,rschronicle,60smusic,2017,"[-0.16608699999999998, -0.337065, 0.26415, 0.0...","[0.061588, -0.428184, 0.038346, -0.121654, 0.2...",0
17023,playrust,newsokuvip,2017,"[-0.587703, -0.768546, 1.3307440000000001, 0.2...","[-0.110479, -0.752474, 0.145984, -0.200932, 0....",0
17024,raredeals,daniellefanclub,2017,"[-0.025138999999999998, -0.089604, 0.085915999...","[-0.010523000000000001, -0.043867, 0.054473, -...",0


# Train and test classifier

In [29]:
X_train = []

for index, row in tqdm(train_all.iterrows()):
    X_train.append(row['source_embed'] + row['target_embed'])
    
X_test = []

for index, row in tqdm(test_all.iterrows()):
    X_test.append(row['source_embed'] + row['target_embed'])

63746it [00:09, 6876.13it/s]
34052it [00:04, 7608.35it/s]


In [30]:
clf = MLPClassifier(hidden_layer_sizes=(3,100),
                    random_state=5,
                    verbose=True,
                    learning_rate_init=0.01)

In [31]:
y_train = train_all['label'].tolist()
clf.fit(X_train,y_train)

Iteration 1, loss = 0.48649184
Iteration 2, loss = 0.45552310
Iteration 3, loss = 0.44985295
Iteration 4, loss = 0.44853592
Iteration 5, loss = 0.44718989
Iteration 6, loss = 0.44572447
Iteration 7, loss = 0.44530293
Iteration 8, loss = 0.44554211
Iteration 9, loss = 0.44349247
Iteration 10, loss = 0.44415124
Iteration 11, loss = 0.44237828
Iteration 12, loss = 0.44215028
Iteration 13, loss = 0.44113027
Iteration 14, loss = 0.44130675
Iteration 15, loss = 0.44112568
Iteration 16, loss = 0.44028796
Iteration 17, loss = 0.43991542
Iteration 18, loss = 0.44006570
Iteration 19, loss = 0.43935143
Iteration 20, loss = 0.43881163
Iteration 21, loss = 0.43900157
Iteration 22, loss = 0.43830664
Iteration 23, loss = 0.43724769
Iteration 24, loss = 0.43789402
Iteration 25, loss = 0.43699045
Iteration 26, loss = 0.43687829
Iteration 27, loss = 0.43702842
Iteration 28, loss = 0.43661378
Iteration 29, loss = 0.43625381
Iteration 30, loss = 0.43618486
Iteration 31, loss = 0.43578889
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(3, 100), learning_rate='constant',
              learning_rate_init=0.01, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=5, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [32]:
ypred = clf.predict(X_test)

In [33]:
print('AUROC:', roc_auc_score(test_all['label'].tolist(), ypred))
print('AUPRC:', average_precision_score(test_all['label'].tolist(), ypred))

AUROC: 0.68736050745918
AUPRC: 0.6509512405822908


# Random Shuffling

In [34]:
def crazyshuffle(arr):
    
    x, y = arr.shape
    rows = np.indices((x,y))[0]
    cols = [np.random.permutation(y) for _ in range(x)]
    
    return arr[rows, cols]

In [35]:
X_train_shuffle = np.array(X_train)
X_train_shuffle = crazyshuffle(X_train_shuffle)

clf_shuffled = MLPClassifier(hidden_layer_sizes=(3,100),
                    random_state=5,
                    verbose=True,
                    learning_rate_init=0.01)

clf_shuffled.fit(X_train_shuffle,y_train)


Iteration 1, loss = 0.61905792
Iteration 2, loss = 0.58229333
Iteration 3, loss = 0.56804246
Iteration 4, loss = 0.55923059
Iteration 5, loss = 0.55392530
Iteration 6, loss = 0.54992245
Iteration 7, loss = 0.54652247
Iteration 8, loss = 0.54403241
Iteration 9, loss = 0.54153082
Iteration 10, loss = 0.54005007
Iteration 11, loss = 0.53806989
Iteration 12, loss = 0.53710108
Iteration 13, loss = 0.53566606
Iteration 14, loss = 0.53481345
Iteration 15, loss = 0.53394138
Iteration 16, loss = 0.53343429
Iteration 17, loss = 0.53292973
Iteration 18, loss = 0.53244418
Iteration 19, loss = 0.53146066
Iteration 20, loss = 0.53058961
Iteration 21, loss = 0.53041396
Iteration 22, loss = 0.52983647
Iteration 23, loss = 0.52887346
Iteration 24, loss = 0.52894175
Iteration 25, loss = 0.52838466
Iteration 26, loss = 0.52739387
Iteration 27, loss = 0.52788855
Iteration 28, loss = 0.52720331
Iteration 29, loss = 0.52693891
Iteration 30, loss = 0.52669058
Iteration 31, loss = 0.52662324
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(3, 100), learning_rate='constant',
              learning_rate_init=0.01, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=5, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [36]:
ypred_shuffle = clf_shuffled.predict(X_test)

print('AUROC:', roc_auc_score(test_all['label'].tolist(), ypred_shuffle))
print('AUPRC:', average_precision_score(test_all['label'].tolist(), ypred_shuffle))

AUROC: 0.6051626923528721
AUPRC: 0.5621079740872794


# Random features

In [37]:
X_train_rand = np.random.rand(np.array(X_train).shape[0],np.array(X_train).shape[1])

clf_rand = MLPClassifier(hidden_layer_sizes=(3,100),
                    random_state=5,
                    verbose=True,
                    learning_rate_init=0.01)

clf_rand.fit(X_train_rand,y_train)

ypred_rand = clf_rand.predict(X_test)

print('AUROC:', roc_auc_score(test_all['label'].tolist(), ypred_rand))
print('AUPRC:', average_precision_score(test_all['label'].tolist(), ypred_rand))

Iteration 1, loss = 0.69348418
Iteration 2, loss = 0.69336403
Iteration 3, loss = 0.69335325
Iteration 4, loss = 0.69329282
Iteration 5, loss = 0.69331584
Iteration 6, loss = 0.69325841
Iteration 7, loss = 0.69326523
Iteration 8, loss = 0.69328682
Iteration 9, loss = 0.69327950
Iteration 10, loss = 0.69328481
Iteration 11, loss = 0.69322287
Iteration 12, loss = 0.69324110
Iteration 13, loss = 0.69327660
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
AUROC: 0.5144191236931751
AUPRC: 0.5131790304244535


# Davis-Bouldin Score

In [38]:
k_clusters = 5

attributes_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(X_train)
print(davies_bouldin_score(X_train,attributes_kmeans.labels_))
      
shuffle_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(X_train_shuffle)
print(davies_bouldin_score(X_train_shuffle,shuffle_kmeans.labels_))
      
rand_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(X_train_rand)
print(davies_bouldin_score(X_train_rand,rand_kmeans.labels_))

2.8067248609795206
16.94979196916366
21.3563056683277


In [39]:
k_clusters = 10

attributes_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(X_train)
print(davies_bouldin_score(X_train,attributes_kmeans.labels_))
      
shuffle_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(X_train_shuffle)
print(davies_bouldin_score(X_train_shuffle,shuffle_kmeans.labels_))
      
rand_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(X_train_rand)
print(davies_bouldin_score(X_train_rand,rand_kmeans.labels_))

2.5568337304995765
11.561541025559473
18.053848454020873
